In [1]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array

# Initialize CSV file to log visits
log_file = "C:/Users/Shweta/Downloads/age_gender.csv"
columns = ['Age', 'Gender', 'Time of Visit']
pd.DataFrame(columns=columns).to_csv(log_file, index=False)

# Define the CNN model for age and gender prediction
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Output layer for binary gender classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

age_model = build_model()
gender_model = build_model()

# Load your pretrained model weights here
# age_model.load_weights("path_to_age_model_weights.h5")
# gender_model.load_weights("path_to_gender_model_weights.h5")

def preprocess_image(frame):
    image = cv2.resize(frame, (128, 128))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image / 255.0  # Normalize

def log_visit(age, gender):
    gender_text = "Male" if gender == 1 else "Female"
    time_of_visit = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    new_row = {'Age': age, 'Gender': gender_text, 'Time of Visit': time_of_visit}
    df = pd.DataFrame([new_row])
    df.to_csv(log_file, mode='a', header=False, index=False)

# Start video capture
cap = cv2.VideoCapture(0)  # Use 0 for webcam; replace with video file path if needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess and make predictions
    face_image = preprocess_image(frame)
    
    # Dummy age and gender predictions for illustration
    predicted_age = np.random.randint(18, 80)  # Replace with actual model prediction
    gender_pred = gender_model.predict(face_image)
    predicted_gender = np.argmax(gender_pred)

    # Display on the frame
    label = f"Age: {predicted_age}, Gender: {'Male' if predicted_gender == 1 else 'Female'}"
    color = (0, 255, 0) if predicted_age <= 60 else (0, 0, 255)
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Log visit if senior
    if predicted_age > 60:
        log_visit(predicted_age, predicted_gender)

    cv2.imshow("Age and Gender Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\Shweta\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
